# Model Selection 모듈

- Train/Test 데이터를 분리하지 않고 머신러닝 수행

In [3]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [5]:
iris = load_iris()
dt_clf = DecisionTreeClassifier()
dt_clf.fit(iris.data, iris.target)    # 분리하지않고 학습하기때문에 fit 사용

DecisionTreeClassifier()

In [7]:
pred = dt_clf.predict(iris.data)
accuracy_score(iris.target, pred)

1.0

- cross_validate() Method

In [8]:
from sklearn.model_selection import cross_validate

In [9]:
dtc = DecisionTreeClassifier()
cross_validate(dtc, iris.data, iris.target)

{'fit_time': array([0.00199151, 0.00099349, 0.        , 0.00099683, 0.00099826]),
 'score_time': array([0.        , 0.        , 0.00099707, 0.        , 0.        ]),
 'test_score': array([0.96666667, 0.96666667, 0.9       , 1.        , 1.        ])}

In [10]:
dtc = DecisionTreeClassifier()
cross_validate(dtc, iris.data, iris.target, return_train_score=True)    # return_train_Score=True 제약조건을 주지않았기때문에 다 나온다.....? 제대로 못들음....

{'fit_time': array([0.        , 0.00104594, 0.0011282 , 0.00096083, 0.        ]),
 'score_time': array([0.00099802, 0.        , 0.        , 0.        , 0.0009973 ]),
 'test_score': array([0.96666667, 0.96666667, 0.9       , 1.        , 1.        ]),
 'train_score': array([1., 1., 1., 1., 1.])}

- Train/Test 데이터 셋을 분리

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, train_size=0.2, random_state=2021
)

In [13]:
import pandas as pd
pd.Series(y_test).value_counts()

0    41
1    41
2    38
dtype: int64

- Stratified 분리

In [14]:
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, train_size=0.2, random_state=2021,
    stratify=iris.target
)           # y값에 따른 분포
pd.Series(y_test).value_counts()

0    40
1    40
2    40
dtype: int64

- cross_val_score() Method

In [15]:
from sklearn.model_selection import cross_val_score

In [16]:
# 성능 지표는 정확도(accuracy), 교차검증 세트는 5개
cross_val_score(dtc, iris.data, iris.target, cv=5)

array([0.96666667, 0.96666667, 0.9       , 1.        , 1.        ])

In [17]:
import numpy as np
scores = cross_val_score(dtc, iris.data, iris.target, cv=5)
np.mean(scores)

0.9666666666666668

### GridSearchCV
- 교차 검증과 최적 하이퍼 파라메터 튜닝을 한꺼번에 수행
- 제일 많이 사용하고 활용도가 높음

In [21]:
dtc = DecisionTreeClassifier(random_state=2021)

In [24]:
# parameter를 dictionary 형태로 설정
params = {
    'max_depth': [2, 3, 4, 5],
    'min_samples_split': [2, 3]
}   # depth에 제한을 안주면 무한정으로 내려가버림 ,그래서 제약조건을 줌

In [25]:
from sklearn.model_selection import GridSearchCV
grid_dtc = GridSearchCV(dtc, param_grid=params, cv=3)

In [26]:
grid_dtc.fit(X_train, y_train)      # 이게 estimate라고 가정하고 구한다...

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=2021),
             param_grid={'max_depth': [2, 3, 4, 5],
                         'min_samples_split': [2, 3]})

In [31]:
grid_dtc.cv_results_

{'mean_fit_time': array([0.00170159, 0.00082517, 0.00066511, 0.00099715, 0.00066471,
        0.00066495, 0.0009973 , 0.00033243]),
 'std_fit_time': array([4.26138291e-04, 2.00184830e-04, 4.70302885e-04, 2.24783192e-07,
        4.70021816e-04, 4.70190252e-04, 1.94667955e-07, 4.70134046e-04]),
 'mean_score_time': array([0.00041779, 0.00034571, 0.00033267, 0.00033236, 0.00033251,
        0.        , 0.        , 0.00066495]),
 'std_score_time': array([0.00059084, 0.0004889 , 0.00047047, 0.00047002, 0.00047025,
        0.        , 0.        , 0.00047019]),
 'param_max_depth': masked_array(data=[2, 2, 3, 3, 4, 4, 5, 5],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_split': masked_array(data=[2, 3, 2, 3, 2, 3, 2, 3],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2, 'min_samples_split'

In [27]:
# 최적 파라미터
grid_dtc.best_params_

{'max_depth': 2, 'min_samples_split': 2}

In [28]:
# 최고 정확도
grid_dtc.best_score_

0.9666666666666667

In [30]:
# 최고 정확도를 가지는 최적 파라미터로 학습한 estimator
best_estimator = grid_dtc.best_estimator_
best_estimator.score(X_test, y_test)

0.9583333333333334